In [3]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [4]:
class Multiclass:
    def __init__(self,input_size,hidden_size,output_size,learning_rate):
        self.input_size=input_size
        self.output_size=output_size
        self.hidden_size=hidden_size
        self.learning_rate=learning_rate
        #Initialize weights and bias
        self.Vij=np.random.uniform(-0.5,0.5,(self.input_size,self.hidden_size))
        self.Wjk=np.random.uniform(-0.5,0.5,(self.hidden_size,self.output_size))
        self.Vbj=np.random.uniform(-0.5,0.5,(1,self.hidden_size))
        self.Wbk=np.random.uniform(-0.5,0.5,(1,self.output_size))

    

    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        return np.where(x > 0, 1, 0)
    
    def forward_propogation(self,x):
            self.hidden_input=np.dot(x,self.Vij)+self.Vbj
            self.hidden_output=self.relu(self.hidden_input)
            self.output_input=np.dot(self.hidden_output,self.Wjk) + self.Wbk
            self.output_final=self.relu(self.output_input)
            return self.output_final
    def backword_propogation(self,x,y,output):
        error=y-output
        #Weight correction in weights
        delta_K=error * self.relu_derivative(output)#Output layer error
        delta_in=delta_K.dot(self.Wjk.T)#Hidden Unit error
        delta_J=delta_in * self.relu_derivative(self.hidden_output)#Hidden Layer error


        # Weigth adjustent in  Weight vector between Output and hidden layer
        self.Wjk+=self.hidden_output.T.dot(delta_K) * self.learning_rate

        #Change in bias between output and hidden layer
        self.Wbk+=np.sum(delta_K)*self.learning_rate
        
        # Weigth adjustent in  Weight vector between input and hidden layer
        self.Vij+=x.T.dot(delta_J) * self.learning_rate

        #Change in bias between input and hidden layer 
        self.Vbj+=np.sum(delta_J)*self.learning_rate
    def train_model(self,x,y,epochs):
        for epoch in range(epochs):
            output=self.forward_propogation((x))
            self.backword_propogation(x,y,output)
            if epoch % 1000 ==0 :
                print(f'Epoch{epoch}:Error{np.mean(np.abs(y-output))}')
    

In [5]:
data = datasets.load_iris()
x = data.data
y = data.target.reshape(-1, 1)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train)

In [7]:
input_size = x_train.shape[1]
hidden_size= 100  # Set the number of neurons in hidden layer
output_size = len(np.unique(y))  # More than 1 neuron in the output layer
learning_rate = 0.1  # Adjust the learning rate as needed
epochs = 10000  # Adjust the number of epochs as needed


In [8]:

ann = Multiclass(input_size, hidden_size, output_size, learning_rate)
ann.train_model(x_train, y_train_encoded, epochs)

Epoch0:Error1.3305666355811094
Epoch1000:Error0.3333333333333333
Epoch2000:Error0.3333333333333333
Epoch3000:Error0.3333333333333333
Epoch4000:Error0.3333333333333333
Epoch5000:Error0.3333333333333333
Epoch6000:Error0.3333333333333333
Epoch7000:Error0.3333333333333333
Epoch8000:Error0.3333333333333333
Epoch9000:Error0.3333333333333333


In [9]:
output = ann.forward_propogation(x_test)
predicted_labels = np.argmax(output, axis=1)
accuracy = np.mean(predicted_labels == y_test.flatten())
print("Accuracy:", accuracy*100,"%")

Accuracy: 33.33333333333333 %
